## 데이터 전처리

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
# 데이터셋 불러오기
breakdown = pd.read_csv('서울시 공공자전거 고장신고 내역_22.01-06.csv', encoding='cp949')
rental = pd.read_csv('서울특별시 공공자전거 대여이력 정보_22.06.csv', encoding='cp949')
location = pd.read_csv('3. 공공자전거 대여소 정보(22.06월 기준).csv', encoding='cp949')

# 등록일시 string -> datetime 변수로 변환
breakdown['등록일시_date'] = pd.to_datetime(breakdown['등록일시']).dt.date

# 6월 고장신고 내역 추출
start = datetime.strptime('2022-06-01','%Y-%m-%d').date()
end = datetime.strptime('2022-06-30','%Y-%m-%d').date()
breakdown = breakdown[(breakdown['등록일시_date']>=start)&(breakdown['등록일시_date']<=end)]

# 복수개의 고장구분 -> 고장신고 1회로 처리
breakdown_unique = breakdown.drop_duplicates(subset = ['자전거번호', '등록일시'])
breakdown_unique.reset_index(drop=True)

df_merge = pd.merge(rental, breakdown_unique, on='자전거번호', how='inner')

In [3]:
df_merge.describe()

,대여 대여소번호,대여거치대,반납대여소번호,반납거치대,이용시간,이용거리
count,518100.000000,518100.000000,518100.000000,518100.000000,518100.000000,518100.000000
mean,1455.945856,0.999745,427.264063,0.994968,23.768664,2796.222322
std,1553.853305,9.898256,199.637664,9.874820,27.438714,3376.616809
min,3.000000,0.000000,3.000000,0.000000,0.000000,0.000000
25%,338.000000,0.000000,247.000000,0.000000,7.000000,887.360000
50%,633.000000,0.000000,421.000000,0.000000,13.000000,1620.000000
75%,3010.000000,0.000000,592.000000,0.000000,31.000000,3272.380000
max,9999.000000,99.000000,770.000000,99.000000,1142.000000,88177.900000


In [4]:
# 자전거번호, 반납일시, 등록일시 기준으로 정렬 후 필요한 변수만 남기기
df = df_merge.sort_values(['자전거번호','반납일시','등록일시']).iloc[:,[0,5,6,7,11]].reset_index(drop=True)

# 반납일시/등록일시 string -> datetime 변수로 변환
df['반납일시'] = pd.to_datetime(df['반납일시'])
df['등록일시'] = pd.to_datetime(df['등록일시'])

# 고장신고 이전 반납이력만 남기기
df_norm = df[(df['반납일시']<=df['등록일시'])]

# 등록일시 직전 반납일시만 남기기
df_final = df_norm.loc[df_norm.groupby(['자전거번호','등록일시'])['반납일시'].idxmax()].reset_index(drop=True)

# 반납대여소번호 별로 묶고 반납대여소별 고장횟수 칼럼 만들기 -> 엑셀로 저장
df_count = pd.DataFrame(df_final.groupby(['반납대여소번호']).count().iloc[:,0])
df_count.columns=['고장횟수']
df_count.to_excel('반납대여소별_고장횟수.xlsx')

In [5]:

# 대여소 정보 데이터셋
location = location.iloc[4:,[0,1,2,3,4,5]]
location = location.rename(columns={'대여소\n번호':'반납대여소번호','Unnamed: 3':'주소','Unnamed: 4':'위도','Unnamed: 5':'경도'})
location['반납대여소번호'] = location['반납대여소번호'].astype(int)
location['위도'] = location['위도'].astype(float)
location['경도'] = location['경도'].astype(float)

# 대여소 정보와 반납대여소별 고장횟수 데이터프레임 inner join
df_inter = pd.merge(df_count, location, on='반납대여소번호', how='inner')

# 조사 대상 3개구(동대문구, 종로구, 중구)만 남기기
conditions = (df_inter['소재지(위치)']=='동대문구')|(df_inter['소재지(위치)']=='종로구')|(df_inter['소재지(위치)']=='중구')
df_inter = df_inter.loc[conditions,['반납대여소번호','고장횟수','소재지(위치)','위도','경도']].reset_index(drop=True)
df_inter['반납대여소번호']=df_inter['반납대여소번호'].astype(int).astype(str)

In [6]:
df_inter

,반납대여소번호,고장횟수,소재지(위치),위도,경도
0,300,7,중구,37.568050,126.969231
1,301,7,종로구,37.575794,126.971451
2,302,24,종로구,37.575947,126.974060
3,303,19,종로구,37.571770,126.974663
4,307,9,종로구,37.570000,126.971100
...,...,...,...,...,...
174,677,14,동대문구,37.580151,127.045799
175,678,35,동대문구,37.576557,127.074310
176,679,1,동대문구,37.577877,127.061684
177,680,43,동대문구,37.592384,127.057411


In [7]:
df_inter.groupby(['소재지(위치)']).sum()

,고장횟수,위도,경도
소재지(위치),,,
동대문구,1580,2442.731110,8258.382973
종로구,791,2254.575401,7619.311127
중구,596,2028.372917,6857.549439


## 데이터 시각화

In [8]:
#구글맵 이용하기 위해 folium 라이브러리 
import folium

#지도의 중심 지정 : 전체 반납장소 위치의 위도, 경도 평균
lat=df_inter['위도'].mean()
lon=df_inter['경도'].mean()
m=folium.Map([lat,lon],zoom_start=12,tiles='cartodbpositron')  

from folium.plugins import MarkerCluster

marker_cluster=MarkerCluster().add_to(m)

for lat, long in zip(df_inter['위도'],df_inter['경도']):
    folium.Marker([lat,long],icon=folium.Icon(color='green')).add_to(marker_cluster)

In [9]:
m

In [11]:
#CircleMarker : 고장횟수 원으로 표현 (많을수록 원의 크기가 크다 )
import json
for i in range(0,len(df_inter)):
    latitude = df_inter.iloc[i]['위도']
    longitude = df_inter.iloc[i]['경도']
    folium.CircleMarker(location=[latitude,longitude], radius=int(df_inter.iloc[i]['고장횟수']/3), 
                        popup="CircleMarker popup",color='#3186cc',fill_color='#3186cc',tooltip="CircleMarker tooltip").add_to(m)
    
folium.LayerControl(collapsed=False).add_to(m)

m